<a href="https://colab.research.google.com/github/nickchak21/QuarkGluonClassifiers/blob/master/EMD_tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install energyflow
!pip install h5py

     |████████████████████████████████| 870kB 10.4MB/s 
     |████████████████████████████████| 2.9MB 29.9MB/s 
  Found existing installation: h5py 2.8.0
    Uninstalling h5py-2.8.0:
      Successfully uninstalled h5py-2.8.0
ERROR: Could not find a version that satisfies the requirement ot (from versions: none)
ERROR: No matching distribution found for ot


In [0]:
!python --version

Python 3.6.8


In [0]:
!pip install POT

     |████████████████████████████████| 307kB 9.8MB/s 


In [0]:
!python -c "import energyflow; energyflow.utils.get_examples()"


Summary of examples:
pfn_example.py exists at /root/.energyflow/examples
dnn_example.py exists at /root/.energyflow/examples
cnn_example.py exists at /root/.energyflow/examples
efn_example.py exists at /root/.energyflow/examples
efp_example.py exists at /root/.energyflow/examples



In [0]:
import numpy as np
# I really don't like matplotlib for histograms.
%matplotlib inline
import matplotlib 
import matplotlib.pyplot as plt


#%jsroot on
from energyflow.emd import emd, emds
from energyflow.datasets import qg_jets
from energyflow.utils import sum_ptyphipids
import ot

In [0]:
from sklearn.metrics import roc_auc_score, roc_curve

In [0]:
from ot.lp import emd_c, check_result
from scipy.spatial.distance import _distance_wrap

In [0]:
# data controls.
# we will use the n nearest neighbors in the "electorate" to vote on what kind of jet each candidate is.
candidates, electorate = 200, 2000

In [0]:
# load quark and gluon jets from Pythia. We want candidates + electorate of *each.*
# Quarks are labeled "1" in this dataset, gluons are "0"

# Shapes of jet,label are: (num_data,max_num_particles,4) and (num_data).
# coordinates are:  (pt,y,phi,pid)
# pad=False makes array of objects, not numbers.
# Load extrajets to be sure we get enough of each type.
extrajets = candidates
jets,labels = qg_jets.load(2*(candidates + electorate + extrajets), generator='pythia', pad=False)
maxevts = candidates+electorate
nprint = int(maxevts*maxevts/10)

# the jet radius for these jets
jetR = 0.4

In [0]:
print(labels.shape)
print(labels)

(4800,)
[1. 1. 1. ... 0. 1. 0.]


In [0]:
Gs = []
Qs = []

for arr,events in [(Gs, jets[labels==0]), (Qs, jets[labels==1])]:
    for i,x in enumerate(events):
        if i >= maxevts:  break
        # ignore padded particles (not used) and remove particle id information(:3)
        x = x[x[:,0] > 0,:3]
        # center jet according to pt-centroid
        yphi_avg = np.average(x[:,1:3], weights=x[:,0], axis=0)
        x[:,1:3] -= yphi_avg
        # mask out any particles farther than R=0.4 away from center (rare)
        x = x[np.linalg.norm(x[:,1:3], axis=1) <= jetR]
        # add to list
        arr.append(x)
        pass #evts
    pass #gluonquark

len(Gs), len(Qs)

(2200, 2200)

In [0]:
# Also look at the original jet properties
# output is a tuple of two arrays of jet [pt,y,phi,m] (vector sum of constituent four-momenta): (quark-jet, gluon-jet) from Monte Carlo simulation (MC)
MC = np.array([sum_ptyphipids(x, error_on_unknown=False) for x in jets[labels==0]]), np.array([sum_ptyphipids(x, error_on_unknown=False) for x in jets[labels==1]])

# simple lists of the rapidity (y) for Gluons and Quarks 
Gy = MC[0][:,1][:maxevts]
Qy = MC[1][:,1][:maxevts]

# simple lists of the transverse momentum (pt) for Gluons and Quarks 
Gpt = MC[0][:,0][:maxevts]
Qpt = MC[1][:,0][:maxevts]


In [0]:
constituentList = Gs[candidates:]+Qs[candidates:]
len(constituentList)

4000

In [0]:
#array of distances of gluon candidate from all constituents. shape is (candidates,constituents)
costM_g  = emds(Gs[0:candidates], constituentList, R=jetR, norm=True, verbose=1, n_jobs=None, print_every=nprint)
#ditto for quarks.
costM_q  = emds(Qs[0:candidates], constituentList, R=jetR, norm=True, verbose=1, n_jobs=None, print_every=nprint)

Processed 4200 events for asymmetric EMD computation in 0.058s
Using 2 worker processes:
  Computed 484000 EMDs, 60.50% done in 102.89s
  Computed 800000 EMDs, 100.00% done in 175.87s
Processed 4200 events for asymmetric EMD computation in 0.047s
Using 2 worker processes:
  Computed 484000 EMDs, 60.50% done in 67.87s
  Computed 800000 EMDs, 100.00% done in 111.98s


In [0]:
indicesG=np.argsort(costM_g)
indicesQ=np.argsort(costM_q)
print(indicesG.shape)
print(indicesG)

(200, 4000)
[[ 725 1504 3739 ...  466 1273 2116]
 [ 116 1769 1912 ... 2004 3070 2427]
 [1790   97 1252 ... 3070 1273 1412]
 ...
 [3917 1927 3855 ... 1383 1975 2995]
 [3892 1656 1833 ... 2995  974 3070]
 [ 314 1145  866 ... 3070 2004 2427]]


In [0]:
h_Gten_array = []
h_Qten_array = []
h_Gtwt_array = []
h_Qtwt_array = []

nVoters = 10
for i in range(candidates): h_Gten_array.append(sum(indicesG[i,:nVoters] < electorate)/nVoters)
for i in range(candidates): h_Qten_array.append(sum(indicesQ[i,:nVoters] < electorate)/nVoters)
nVoters = 20
for i in range(candidates): h_Gtwt_array.append(sum(indicesG[i,:nVoters] < electorate)/nVoters)  
for i in range(candidates): h_Qtwt_array.append(sum(indicesQ[i,:nVoters] < electorate)/nVoters)

In [0]:
print(h_Gten_array)
print(h_Qten_array)

[0.6, 0.4, 0.7, 1.0, 0.5, 0.4, 0.8, 0.5, 0.6, 0.7, 0.7, 0.9, 0.6, 0.4, 1.0, 0.6, 0.4, 0.9, 0.6, 0.6, 0.7, 0.7, 0.6, 0.6, 1.0, 0.3, 0.9, 0.9, 0.0, 0.5, 0.7, 0.2, 0.2, 0.7, 0.6, 0.6, 0.2, 1.0, 0.1, 0.5, 0.4, 0.3, 0.4, 1.0, 0.7, 0.6, 0.8, 0.3, 0.3, 0.8, 0.9, 0.4, 0.7, 0.1, 0.8, 0.8, 0.6, 0.9, 0.8, 0.4, 0.6, 0.9, 0.7, 0.0, 0.7, 0.8, 0.7, 0.8, 0.3, 0.7, 0.7, 0.9, 0.0, 0.9, 0.6, 0.7, 0.0, 0.9, 0.8, 0.9, 0.8, 0.5, 0.4, 0.8, 0.1, 0.8, 0.2, 0.9, 0.9, 0.5, 0.6, 0.8, 0.4, 0.5, 0.1, 0.7, 0.4, 0.5, 0.6, 0.7, 0.8, 0.7, 1.0, 0.9, 0.7, 0.0, 0.9, 0.9, 0.5, 0.7, 0.8, 0.8, 0.7, 0.2, 0.6, 0.6, 0.3, 0.3, 0.9, 0.7, 0.5, 0.9, 0.8, 0.9, 0.7, 0.6, 1.0, 0.8, 0.0, 0.8, 0.6, 0.3, 0.9, 0.8, 0.6, 0.9, 1.0, 0.8, 0.6, 0.7, 0.6, 0.3, 0.0, 0.9, 0.5, 0.7, 0.6, 0.4, 1.0, 0.1, 0.2, 0.7, 0.7, 0.9, 0.6, 0.8, 0.5, 0.6, 0.6, 0.6, 1.0, 1.0, 0.5, 0.5, 0.8, 0.6, 0.9, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.9, 0.7, 0.9, 0.5, 0.0, 0.6, 1.0, 0.5, 0.8, 0.9, 0.4, 0.9, 0.6, 0.9, 0.9, 0.8, 0.8, 0.3, 1.0, 0.5, 0.5, 0.7, 0.5, 0.3, 0.4, 0.9]

In [0]:
h_Gten_nparray = np.asarray(h_Gten_array)
h_Qten_nparray = np.asarray(h_Qten_array)
h_Gtwt_nparray = np.asarray(h_Gtwt_array)
h_Qtwt_nparray = np.asarray(h_Qtwt_array)

In [0]:
true_y_g = np.ones(200)
true_y_q = np.zeros(200)

true_values = np.concatenate([true_y_g, true_y_q])

ten_predicted = np.concatenate([h_Gten_nparray, h_Qten_nparray])
emd_auc_10 = roc_auc_score(y_true=true_values, y_score=ten_predicted)

In [0]:
print(emd_auc_10)

0.8192999999999999


In [0]:
twt_predicted = np.concatenate([h_Gtwt_nparray, h_Qtwt_nparray])
emd_auc_20 = roc_auc_score(y_true=true_values, y_score=twt_predicted)
print(emd_auc_20)

0.8374375000000001
